# Intento de usar PyVISA

In [1]:
import pyvisa

In [2]:
rm = pyvisa.ResourceManager('@py')

In [3]:
# registro automático de recursos
resourcesFound = rm.list_resources()
type(resourcesFound), resourcesFound[0]

(tuple, 'USB0::5710::4004::TW00048101::0::INSTR')

In [4]:
# apertura automática del recurso USB
inst = rm.open_resource(resourcesFound[0])
print(inst.query("*IDN?"))

Berkeley Nucleonics Corp.,Model 1105,TW00048101,1.2.8-1H3-01-00-01



In [ ]:
inst.write('*RST')

6

## Secuencia configuración y lectura simple
Pruebo con una medida de frecuencia en el canal 1.

In [6]:
# Lectura frecuencia canal 1
inst.write('CONFIGURE:FREQUENCY 1')

23

In [8]:
dato = inst.query(':READ:SCALAR:VOLTAGE:FREQUENCY?')
print(dato)

+5.500E+004



Anduvo bien. Noté que al configurar redujo el número de decimales.

## Intervalo de tiempo entre canales

In [12]:
dato = inst.query(':MEASURE:TINT? 1,2')
print(dato)

+9.91000000000000E+037



In [16]:
inst.write('CONFIGURE:TINTERVAL')

21

In [20]:
dato = inst.query(':READ:SCALAR:TINTERVAL?')
print(dato)

+5.55452E-005



In [21]:
# get current date and time
import datetime
now = datetime.datetime.now()
print(now)

2024-02-19 15:15:36.169946


In [23]:
# take a measuremnt every minute and store into a dataframe
import time
import pandas as pd

df = pd.DataFrame(columns=['time', 'frequency', 'tinterval'])

for i in range(5):
    dato = inst.query(':READ:SCALAR:VOLTAGE:FREQUENCY?')
    dato2 = inst.query(':READ:SCALAR:TINTERVAL?')
    now = datetime.datetime.now()
    # append no longer used, use concat instead
    df = pd.concat([df, pd.DataFrame({'time': [now], 'frequency': [dato], 'tinterval': [dato2]})], ignore_index=True)
    # df = df.append({'time': now, 'frequency': dato, 'tinterval': dato2}, ignore_index=True)
    time.sleep(60)

df  


/tmp/ipykernel_15213/2473344595.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame({'time': [now], 'frequency': [dato], 'tinterval': [dato2]})], ignore_index=True)


,time,frequency,tinterval
0,2024-02-19 15:18:08.662832,+9.91000000000000E+037\n,+8.333245E-004\n
1,2024-02-19 15:19:10.874597,+9.91000000000000E+037\n,+8.333248E-004\n
2,2024-02-19 15:20:13.031225,+9.91000000000000E+037\n,+8.333248E-004\n
3,2024-02-19 15:21:15.241679,+9.91000000000000E+037\n,+8.333246E-004\n
4,2024-02-19 15:22:17.428726,+9.91000000000000E+037\n,+8.333248E-004\n


In [24]:
# save to file in csv format the df
df.to_csv('./data.csv')

In [25]:
1/10E9

1e-10